### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
appName = "Lap Times Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
qualifying_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/qualifying")

### Step 2 - Rename and add new columns

In [ ]:
qualifying_column_mapping = {
    "qualifyId": "qualify_id",
    "raceId": "race_id",
    "driverId": "driver_id"
}

qualifying_df = rename_columns(qualifying_df, qualifying_column_mapping)


In [ ]:
qualifying_df = add_ingestion_date(qualifying_df)

### Step 3 - Write a data to a Datalake as a Parquet

In [ ]:
qualifying_df.write.mode("overwrite").parquet(f"{processed_folder_path}/qualifying")